In [1]:
import h5py
import os
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

In [2]:
code_path = "../"
code_path

'../'

In [3]:
import sys
sys.path.append(code_path)

from encoding import *
from preprocessing import get_ordered_representations, normalize_train_test, concat_past_features, lanczosinterp2D, delete_block_edges, shuffle_words
from encoding import nested_blocked_cv, ridge_regression_fit_sklearn, ridge_regression_predict_torch
from analysis import pearson_correlation

All tests passed for pearson_correlation function.


In [4]:
LLM_input_sequence = [np.str_('Harry'),
 np.str_('had'),
 np.str_('never'),
 np.str_('believed'),
 np.str_('he'),
]

# LLM_input_sequence = [np.str_('Harry'),
#  np.str_('had'),
#  np.str_('never'),
#  np.str_('believed'),
#  np.str_('he'),
#  np.str_('would'),
#  np.str_('meet'),
#  np.str_('a'),
#  np.str_('boy'),
#  np.str_('he'),
#  np.str_('hated'),
#  np.str_('more'),
#  np.str_('than'),
#  np.str_('Dudley,'),
#  np.str_('but'),
#  np.str_('that'),
#  np.str_('was'),
#  np.str_('before'),
#  np.str_('he'),
#  np.str_('met'),
#  np.str_('Draco'),
#  np.str_('Malfoy.'),]

In [5]:
dict_LLM_input_sequence = {i: word for i, word in enumerate(LLM_input_sequence)}
dict_LLM_input_sequence

{0: np.str_('Harry'),
 1: np.str_('had'),
 2: np.str_('never'),
 3: np.str_('believed'),
 4: np.str_('he')}

In [35]:
shuffled_sequence, inverse = shuffle_words(LLM_input_sequence, percentage=1.)

In [36]:
shuffled_sequence

[np.str_('had'),
 np.str_('believed'),
 np.str_('never'),
 np.str_('he'),
 np.str_('Harry')]

In [37]:
dict_shuffled_sequence = {i: word for i, word in zip(inverse, shuffled_sequence)}
dict_shuffled_sequence

{1: np.str_('had'),
 3: np.str_('believed'),
 2: np.str_('never'),
 4: np.str_('he'),
 0: np.str_('Harry')}

In [38]:
inverse

[1, 3, 2, 4, 0]

In [40]:
idx_mapping = {i: j for i, j in zip(range(len(LLM_input_sequence)), inverse)}
idx_mapping

{0: 1, 1: 3, 2: 2, 3: 4, 4: 0}

# Load model

In [41]:
# Set to a writable location, like /tmp or your home dir
HF_home = "../../cache"  # or "/home/youruser/.cache/huggingface"
os.makedirs(HF_home, exist_ok=True)

In [42]:
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, cache_dir=HF_home)
model = AutoModel.from_pretrained(model_name, cache_dir=HF_home).to("cpu")
print("Loaded model and tokenizer.")

Loaded model and tokenizer.


In [43]:
layer_idx = -7

## Forwarding original words to the model

In [24]:
prompt_text = " ".join(LLM_input_sequence)
prompt_text

'Harry had never believed he'

In [25]:
# compute representations 
occurrences, representations = get_ordered_representations(
    ordered_strings_to_find=strings_to_find,
    prompt=prompt_text,
    tokenizer=tokenizer,
    model=model
)
print("Computed word level representations for the input sequence.")

Computed word level representations for the input sequence.


In [26]:
original_layer_representations = representations[layer_idx]
original_layer_representations.shape

torch.Size([5, 2048])

## Forward shuffled words to the model

In [27]:
prompt_text = " ".join(shuffled_sequence)
prompt_text

'Harry had never believed he'

In [28]:
# compute representations 
occurrences, representations = get_ordered_representations(
    ordered_strings_to_find=strings_to_find,
    prompt=prompt_text,
    tokenizer=tokenizer,
    model=model
)
print("Computed word level representations for the input sequence.")

Computed word level representations for the input sequence.


In [29]:
shuffled_layer_representations = representations[layer_idx]
shuffled_layer_representations.shape

torch.Size([5, 2048])